# Notebook 21: Complete System Validation (Quali + Race)

**Two-stage prediction system:**
1. Predict qualifying positions
2. After quali, predict race outcomes (position, podium, points, DNF)

## Metrics Tracked:
**QUALIFYING:**
- MAE (same as before)
- Calibration (1σ, 2σ)

**RACE:**
- Position MAE (in positions)
- Podium accuracy (%)
- Points accuracy (%)
- DNF Brier score (lower = better)

## Test Season:
2024 → 2025 (faster validation, ~30 minutes)

In [ ]:
# CLEAN ERROR HANDLING
import warnings
import logging
import sys

warnings.filterwarnings('ignore')
logging.getLogger('fastf1').setLevel(logging.ERROR)
logging.getLogger('requests').setLevel(logging.ERROR)
logging.getLogger('urllib3').setLevel(logging.ERROR)
logging.getLogger('requests_cache').setLevel(logging.ERROR)
sys.tracebacklimit = 0

In [ ]:
# Imports
import json
import numpy as np
import pandas as pd
import fastf1 as ff1
from pathlib import Path
import copy
from datetime import datetime
import time

ff1.Cache.enable_cache('../data/raw/.fastf1_cache')

In [ ]:
# Configuration (using tuned parameters from 20B)
TUNED_PARAMS = {
    'initial_uncertainty': 0.20,
    'min_uncertainty': 0.06,
    'measurement_noise': 0.04,
    'driver_specific_mins': {
        'VER': 0.05, 'RUS': 0.05, 'LEC': 0.06, 'HAM': 0.06,
        'NOR': 0.06, 'ALO': 0.06, 'SAI': 0.07, 'PER': 0.10,
        'default': 0.07
    }
}

TEST_SEASON = {'from': 2024, 'to': 2025, 'type': 'stable'}
TRACKED_DRIVERS = ['VER', 'HAM', 'LEC', 'NOR', 'SAI', 'PER', 'RUS', 'ALO']

## Data Extraction (with Racecraft)

In [ ]:
def extract_race_results(year, race_name):
    """Extract both quali AND race results."""
    time.sleep(0.5)
    
    try:
        quali = ff1.get_session(year, race_name, 'Q')
        quali.load(laps=False, telemetry=False, weather=False)
        
        race = ff1.get_session(year, race_name, 'R')
        race.load(laps=False, telemetry=False, weather=False)
        
        results = {}
        
        for _, row in quali.results.iterrows():
            driver = row['Abbreviation']
            quali_pos = row['Position']
            
            if pd.notna(driver) and pd.notna(quali_pos):
                results[driver] = {'quali_pos': int(quali_pos)}
        
        for _, row in race.results.iterrows():
            driver = row['Abbreviation']
            race_pos = row['Position']
            
            dnf = row.dnf if hasattr(row, 'dnf') else False
            status = str(row['Status']) if 'Status' in row else ''
            if not dnf and status:
                dnf = 'Finished' not in status and '+' not in status
            
            if pd.notna(driver) and driver in results:
                if pd.notna(race_pos):
                    results[driver]['race_pos'] = int(race_pos)
                results[driver]['dnf'] = dnf
        
        return results
        
    except Exception as e:
        print(f"    ❌ Error: {type(e).__name__}")
        return None

In [ ]:
def extract_season_characteristics(year):
    """Extract prior year characteristics INCLUDING racecraft."""
    schedule = ff1.get_event_schedule(year)
    driver_stats = {}
    
    print(f"  Extracting {year} season characteristics...")
    
    for _, event in schedule.iterrows():
        if event['EventFormat'] != 'conventional':
            continue
        
        race_name = event['EventName']
        results = extract_race_results(year, race_name)
        if not results:
            continue
        
        for driver, data in results.items():
            if driver not in driver_stats:
                driver_stats[driver] = {
                    'quali_positions': [],
                    'race_positions': [],
                    'positions_gained': [],
                    'dnfs': 0,
                    'races': 0
                }
            
            driver_stats[driver]['quali_positions'].append(data['quali_pos'])
            driver_stats[driver]['races'] += 1
            
            if data.get('dnf', False):
                driver_stats[driver]['dnfs'] += 1
            elif 'race_pos' in data:
                driver_stats[driver]['race_positions'].append(data['race_pos'])
                positions_gained = data['quali_pos'] - data['race_pos']
                driver_stats[driver]['positions_gained'].append(positions_gained)
    
    # Calculate characteristics
    characteristics = {}
    for driver, stats in driver_stats.items():
        if stats['races'] == 0:
            continue
        
        avg_quali_pos = np.mean(stats['quali_positions'])
        avg_pace = 1.0 - (avg_quali_pos - 1) / 19
        dnf_rate = stats['dnfs'] / stats['races']
        
        # Racecraft score from positions gained
        if stats['positions_gained']:
            avg_gain = np.mean(stats['positions_gained'])
            # Scale to 0-1: +3 = 1.0, 0 = 0.5, -3 = 0.0
            racecraft_score = 0.5 + (avg_gain / 6.0)
            racecraft_score = np.clip(racecraft_score, 0.0, 1.0)
        else:
            racecraft_score = 0.5
        
        characteristics[driver] = {
            'avg_quali_pace': float(avg_pace),
            'dnf_rate': float(dnf_rate),
            'racecraft_score': float(racecraft_score),
            'races_completed': stats['races']
        }
    
    print(f"  ✅ Extracted characteristics for {len(characteristics)} drivers")
    return characteristics

## Race Prediction System

In [ ]:
def predict_race_from_quali(actual_quali_results, driver_priors):
    """
    Predict race outcomes based on actual quali results.
    
    Returns:
    - expected_race_pos
    - podium_probability
    - points_probability
    - dnf_probability
    """
    race_predictions = {}
    
    for driver, quali_pos in actual_quali_results.items():
        if driver not in driver_priors['drivers']:
            continue
        
        driver_data = driver_priors['drivers'][driver]
        
        # Get racecraft and DNF data
        racecraft_score = driver_data.get('racecraft', {}).get('skill_score', 0.5)
        dnf_prob = driver_data.get('dnf_risk', {}).get('rate', 0.1)
        quali_uncertainty = driver_data['pace']['uncertainty']
        
        # Racecraft effect: ±3 positions
        racecraft_delta = (racecraft_score - 0.5) * 6
        
        # Expected race position
        expected_pos = quali_pos - racecraft_delta
        expected_pos = np.clip(expected_pos, 1, 20)
        
        # Race uncertainty
        race_uncertainty = np.sqrt(
            (quali_uncertainty * 19)**2 + 3**2
        )
        
        # Position probability distribution
        positions = np.arange(1, 21)
        position_probs = np.exp(-0.5 * ((positions - expected_pos) / race_uncertainty) ** 2)
        position_probs = position_probs / position_probs.sum()
        
        # Adjust for DNF
        position_probs_finish = position_probs * (1 - dnf_prob)
        position_probs_finish[19] += dnf_prob
        
        # Calculate probabilities
        podium_prob = position_probs_finish[:3].sum()
        points_prob = position_probs_finish[:10].sum()
        
        race_predictions[driver] = {
            'quali_pos': quali_pos,
            'expected_race_pos': float(expected_pos),
            'race_uncertainty': float(race_uncertainty),
            'podium_probability': float(podium_prob),
            'points_probability': float(points_prob),
            'dnf_probability': float(dnf_prob)
        }
    
    return race_predictions

## Metrics Calculation

In [ ]:
def calculate_quali_metrics(predictions, uncertainties, actuals):
    """Calculate quali prediction metrics."""
    errors = []
    within_1sigma = []
    within_2sigma = []
    
    for driver in predictions:
        if driver in actuals:
            pred = predictions[driver]
            unc = uncertainties[driver]
            actual = actuals[driver]
            error = abs(pred - actual)
            
            errors.append(error)
            within_1sigma.append(error <= unc)
            within_2sigma.append(error <= 2 * unc)
    
    return {
        'mae': np.mean(errors) if errors else None,
        'calibration_1sigma': sum(within_1sigma) / len(within_1sigma) * 100 if within_1sigma else None,
        'calibration_2sigma': sum(within_2sigma) / len(within_2sigma) * 100 if within_2sigma else None,
        'n': len(errors)
    }


def calculate_race_metrics(predictions, actuals):
    """Calculate race prediction metrics."""
    position_errors = []
    podium_correct = []
    points_correct = []
    dnf_brier = []
    
    for driver, pred in predictions.items():
        if driver not in actuals:
            continue
        
        actual = actuals[driver]
        actual_dnf = actual.get('dnf', False)
        
        # Position error (finishers only)
        if not actual_dnf and 'race_pos' in actual:
            error = abs(pred['expected_race_pos'] - actual['race_pos'])
            position_errors.append(error)
        
        # Podium prediction
        pred_podium = pred['podium_probability'] > 0.5
        actual_podium = (actual.get('race_pos', 21) <= 3) and not actual_dnf
        podium_correct.append(pred_podium == actual_podium)
        
        # Points prediction
        pred_points = pred['points_probability'] > 0.5
        actual_points = (actual.get('race_pos', 21) <= 10) and not actual_dnf
        points_correct.append(pred_points == actual_points)
        
        # DNF prediction (Brier score)
        actual_dnf_binary = 1.0 if actual_dnf else 0.0
        brier = (pred['dnf_probability'] - actual_dnf_binary) ** 2
        dnf_brier.append(brier)
    
    return {
        'position_mae': np.mean(position_errors) if position_errors else None,
        'podium_accuracy': np.mean(podium_correct) * 100 if podium_correct else None,
        'points_accuracy': np.mean(points_correct) * 100 if points_correct else None,
        'dnf_brier_score': np.mean(dnf_brier) if dnf_brier else None,
        'n': len(podium_correct)
    }

## Tuned Bayesian Update (from 20B)

In [ ]:
def tuned_bayesian_update(priors, race_results, season_progress):
    """Bayesian update with tuning (same as 20B)."""
    posteriors = copy.deepcopy(priors)
    posteriors['week'] = priors.get('week', 0) + 1
    posteriors['races_seen'] = priors.get('races_seen', 0) + 1
    
    grid_size = 20
    
    for driver, result in race_results.items():
        if driver not in posteriors['drivers']:
            continue
        
        driver_data = posteriors['drivers'][driver]
        races_seen = driver_data.get('races_seen', 0)
        
        # Adaptive learning
        base_alpha = max(0.05, 1.0 / (races_seen + 2))
        if 0.4 < season_progress < 0.8:
            base_alpha *= 0.7
        alpha = base_alpha
        
        # Update pace
        observed_pace = 1.0 - (result['quali_pos'] - 1) / (grid_size - 1)
        prior_pace = driver_data['pace']['quali_pace']
        new_pace = (1 - alpha) * prior_pace + alpha * observed_pace
        
        driver_data['pace']['quali_pace'] = float(new_pace)
        
        # Store history
        if 'pace_history' not in driver_data:
            driver_data['pace_history'] = []
        driver_data['pace_history'].append(float(observed_pace))
        
        # Update uncertainty with tuning
        error = abs(prior_pace - observed_pace)
        prior_uncertainty = driver_data['pace']['uncertainty']
        
        dnf = result.get('dnf', False)
        
        if dnf:
            obs_var = 0.10
            n_obs = 0.5
        else:
            pace_var = np.var(driver_data['pace_history'][-5:]) if len(driver_data['pace_history']) >= 2 else 0.08
            obs_var = pace_var + TUNED_PARAMS['measurement_noise']
            n_obs = 1.0
        
        # Bayesian update
        prior_var = prior_uncertainty ** 2
        posterior_var = 1.0 / (1.0/prior_var + n_obs/obs_var)
        new_uncertainty = np.sqrt(posterior_var)
        
        # Outlier detection
        if error > 0.5:
            new_uncertainty *= 1.2
        
        # Driver-specific minimum
        driver_min = TUNED_PARAMS['driver_specific_mins'].get(driver, 0.07)
        new_uncertainty = max(new_uncertainty, driver_min)
        
        driver_data['pace']['uncertainty'] = float(new_uncertainty)
        driver_data['races_seen'] = races_seen + 1
    
    return posteriors

## Main Validation

In [ ]:
print("="*70)
print("COMPLETE SYSTEM VALIDATION (QUALI + RACE)")
print("="*70)
print(f"\nTest Season: {TEST_SEASON['from']} → {TEST_SEASON['to']}")
print(f"Tracked Drivers: {', '.join(TRACKED_DRIVERS)}")

In [ ]:
# Extract prior year characteristics
prior_characteristics = extract_season_characteristics(TEST_SEASON['from'])

# Initialize priors with racecraft data
priors = {
    'week': 0,
    'season': TEST_SEASON['to'],
    'races_seen': 0,
    'drivers': {}
}

for driver_code, char in prior_characteristics.items():
    priors['drivers'][driver_code] = {
        'pace': {
            'quali_pace': char['avg_quali_pace'],
            'uncertainty': TUNED_PARAMS['initial_uncertainty'],
            'confidence': 'low'
        },
        'dnf_risk': {'rate': char['dnf_rate']},
        'racecraft': {'skill_score': char['racecraft_score']},
        'races_seen': 0,
        'pace_history': []
    }

In [ ]:
# Run validation
schedule = ff1.get_event_schedule(TEST_SEASON['to'])
current_priors = copy.deepcopy(priors)

results = {
    'quali_metrics_per_race': [],
    'race_metrics_per_race': [],
    'drivers': {driver: {
        'quali_errors': [],
        'race_position_errors': [],
        'podium_predictions': [],
        'points_predictions': []
    } for driver in TRACKED_DRIVERS}
}

week = 0
total_races = len([e for _, e in schedule.iterrows() if e['EventFormat'] == 'conventional'])

print(f"\n{'='*70}")
print("RACE-BY-RACE RESULTS")
print("="*70)

In [ ]:
for _, event in schedule.iterrows():
    if event['EventFormat'] != 'conventional':
        continue
    
    week += 1
    race_name = event['EventName']
    season_progress = week / total_races
    
    print(f"\n📍 Week {week}/{total_races}: {race_name}")
    
    # Extract actual results (both quali and race)
    actual_results = extract_race_results(TEST_SEASON['to'], race_name)
    if not actual_results:
        continue
    
    # STAGE 1: Quali predictions
    quali_predictions = {}
    quali_uncertainties = {}
    quali_actuals = {}
    
    for driver in TRACKED_DRIVERS:
        if driver in current_priors['drivers'] and driver in actual_results:
            pred_pace = current_priors['drivers'][driver]['pace']['quali_pace']
            unc = current_priors['drivers'][driver]['pace']['uncertainty']
            actual_pace = 1.0 - (actual_results[driver]['quali_pos'] - 1) / 19
            
            quali_predictions[driver] = pred_pace
            quali_uncertainties[driver] = unc
            quali_actuals[driver] = actual_pace
    
    quali_metrics = calculate_quali_metrics(quali_predictions, quali_uncertainties, quali_actuals)
    
    # STAGE 2: Race predictions (based on ACTUAL quali)
    actual_quali_only = {d: r['quali_pos'] for d, r in actual_results.items()}
    race_predictions = predict_race_from_quali(actual_quali_only, current_priors)
    race_metrics = calculate_race_metrics(race_predictions, actual_results)
    
    print(f"  📊 Quali: MAE {quali_metrics['mae']:.3f} | Cal {quali_metrics['calibration_1sigma']:.0f}%")
    print(f"  🏁 Race:  MAE {race_metrics['position_mae']:.3f} | Podium {race_metrics['podium_accuracy']:.0f}% | Points {race_metrics['points_accuracy']:.0f}%")
    
    # Store metrics
    results['quali_metrics_per_race'].append(quali_metrics)
    results['race_metrics_per_race'].append(race_metrics)
    
    # Update priors
    current_priors = tuned_bayesian_update(current_priors, actual_results, season_progress)

## Final Summary

In [ ]:
print(f"\n{'='*70}")
print("FINAL SUMMARY")
print("="*70)

# Quali summary
quali_maes = [m['mae'] for m in results['quali_metrics_per_race'] if m['mae'] is not None]
quali_cals = [m['calibration_1sigma'] for m in results['quali_metrics_per_race'] if m['calibration_1sigma'] is not None]

print(f"\n🏎️  QUALIFYING PREDICTIONS:")
print(f"   MAE: {np.mean(quali_maes):.3f}")
print(f"   Calibration (1σ): {np.mean(quali_cals):.1f}%")
print(f"   Races: {len(quali_maes)}")

# Race summary
race_maes = [m['position_mae'] for m in results['race_metrics_per_race'] if m['position_mae'] is not None]
podium_accs = [m['podium_accuracy'] for m in results['race_metrics_per_race'] if m['podium_accuracy'] is not None]
points_accs = [m['points_accuracy'] for m in results['race_metrics_per_race'] if m['points_accuracy'] is not None]
dnf_briers = [m['dnf_brier_score'] for m in results['race_metrics_per_race'] if m['dnf_brier_score'] is not None]

print(f"\n🏁 RACE PREDICTIONS:")
print(f"   Position MAE: ±{np.mean(race_maes):.1f} positions")
print(f"   Podium Accuracy: {np.mean(podium_accs):.1f}%")
print(f"   Points Accuracy: {np.mean(points_accs):.1f}%")
print(f"   DNF Brier Score: {np.mean(dnf_briers):.3f} (lower = better)")
print(f"   Races: {len(race_maes)}")

In [ ]:
print(f"\n{'='*70}")
print("WHAT THIS MEANS:")
print("="*70)
print(f"• Quali predictions: {np.mean(quali_maes):.3f} MAE")
print(f"• Race predictions: ±{np.mean(race_maes):.1f} positions")
print(f"• Podium predictions: {np.mean(podium_accs):.0f}% correct")
print(f"• Points predictions: {np.mean(points_accs):.0f}% correct")
print(f"\nUsers get FULL race weekend predictions!")

## Save Results

In [ ]:
output_path = Path('../data/processed/testing_files/validation')
output_path.mkdir(parents=True, exist_ok=True)

with open(output_path / 'complete_system_validation.json', 'w') as f:
    json.dump({
        'test_season': TEST_SEASON,
        'quali_summary': {
            'mae': float(np.mean(quali_maes)),
            'calibration': float(np.mean(quali_cals))
        },
        'race_summary': {
            'position_mae': float(np.mean(race_maes)),
            'podium_accuracy': float(np.mean(podium_accs)),
            'points_accuracy': float(np.mean(points_accs)),
            'dnf_brier': float(np.mean(dnf_briers))
        },
        'per_race_metrics': {
            'quali': results['quali_metrics_per_race'],
            'race': results['race_metrics_per_race']
        }
    }, f, indent=2)

print(f"\n✅ Results saved to: {output_path / 'complete_system_validation.json'}")